In [0]:
#question 6 - visualisation done!
%pip install bokeh

Python interpreter will be restarted.
  Using cached bokeh-2.4.2-py3-none-any.whl (18.5 MB)
  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
  Using cached typing_extensions-4.2.0-py3-none-any.whl (24 kB)
Python interpreter will be restarted.


In [0]:
#File's directories
ctFile = ("dbfs:/FileStore/tables/clinicaltrial_2021")
mshFile = ("dbfs:/FileStore/tables/mesh.csv")
pharFile = ("dbfs:/FileStore/tables/pharma.csv")

#File's year
ctYear = ('2021')

#File's delimiters
ctDelimiter = ('|')
condDelimiter = (',')
mshDelimiter = (',')
pharDelimiter = (',')
stats = 'Completed'

#Collumns from clienttrial file
typeCol = ('Type')
condCol = ('Conditions')
pcCol = ('Parent_Company')
compCol = ('Completion')

#Collumns from pharma file
sponCol = ('Sponsor')


In [0]:
# 1 - Creates a new dataset (ctDF) using the ctFile

ctDF = spark.read.options(header='True' , delimiter = ctDelimiter).csv(ctFile)

ctDF.show()

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|
+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|NCT02758028|The University of...|          Recruiting|Aug 2005|  Nov 2021|      Interventional|  Apr 2016|                null|                null|
|NCT02751957|     Duke University|           Completed|Jul 2016|  Jul 2020|      Interventional|  Apr 2016|Autistic Disorder...|                null|
|NCT02758483|Universidade Fede...|           Completed|Mar 2017|  Jan 2018|      Interventional|  Apr 2016|   Diabetes Mellitus|                null|
|NCT02759848|Istanbul Medeniye...|           Completed|Jan 2012|  Dec 2014|       Observational|  Ma

In [0]:
# 1 - Counts the number of studies in the datased

ctDF.count()

Out[3]: 387261

In [0]:
typeDf = ctDF.select(typeCol).groupby(typeCol).count().sort('count' , ascending = False).withColumnRenamed('count', 'Frequency')

typeDf.show()

+--------------------+---------+
|                Type|Frequency|
+--------------------+---------+
|      Interventional|   301472|
|       Observational|    77540|
|Observational [Pa...|     8180|
|     Expanded Access|       69|
+--------------------+---------+



In [0]:
# 1 - Gets the top 5 conditions with their frequencies

from pyspark.sql import functions as F

condDF = ctDF.select(F.explode(F.split(F.col(condCol) , condDelimiter))).withColumnRenamed('Col', 'Condition').groupBy('Condition').count().sort('count', ascending = False).withColumnRenamed('count', 'Frequency')

condDF.show(5)

+-----------------+---------+
|        Condition|Frequency|
+-----------------+---------+
|        Carcinoma|    13389|
|Diabetes Mellitus|    11080|
|        Neoplasms|     9371|
| Breast Neoplasms|     8640|
|         Syndrome|     8032|
+-----------------+---------+
only showing top 5 rows



In [0]:
# 1 - Creates meshDF using the mesh dataset
meshDF = spark.read.options(header='True' , delimiter = mshDelimiter).csv(mshFile)
#meshDF.show()

# 2 - Gets the term column and the first 3 characters (roots) of the tree column and rename it to "Roots"
treeDf = meshDF.select('term' , F.substring('tree', 1, 3)).withColumnRenamed('substring(tree, 1, 3)', 'Roots')
#treeDf.show(5)

# 3 - Joins the treeDF with the condDF using the inner join type and matching the column term with the Condition one
joinCondMsh = treeDf.join(condDF, treeDf.term == condDF.Condition, 'inner')

# 4 - Gets the Frequency of the Roots
topRoots = joinCondMsh.select('Roots', 'Frequency').groupBy('Roots').sum('Frequency').withColumnRenamed('sum(Frequency)', 'Frequency').sort('Frequency', ascending = False)

topRoots.show(5)

+-----+---------+
|Roots|Frequency|
+-----+---------+
|  C04|   143994|
|  C23|   136079|
|  C01|   106674|
|  C14|    94523|
|  C10|    92310|
+-----+---------+
only showing top 5 rows



In [0]:
# 1 - Creates pharDF using the pharma dataset
pharDF = spark.read.options(header='True' , delimiter = pharDelimiter).csv(pharFile)

# 2 - Creates pcDF with the Paret_Company comlumn from pharDF
pcDF = pharDF.select(pcCol)

# 3 - Creates sponDF with the Sponsor column from ctDF
sponDF = ctDF.select(sponCol)

# 4 - Left joins sponDF with pcDF matching the Parent_Company colum with the Sponsor column
joinSponPc = sponDF.join(pcDF, pcDF.Parent_Company == sponDF.Sponsor, "left")

# 5 - Filters the null results groups by ******
topSpon = joinSponPc.filter(joinSponPc.Parent_Company.isNull()).groupBy(sponCol).count().sort('count', ascending = False)
topSpon.show(10)

+--------------------+-----+
|             Sponsor|count|
+--------------------+-----+
|National Cancer I...| 3218|
|M.D. Anderson Can...| 2414|
|Assistance Publiq...| 2369|
|         Mayo Clinic| 2300|
|Merck Sharp & Doh...| 2243|
|   Assiut University| 2154|
|Novartis Pharmace...| 2088|
|Massachusetts Gen...| 1971|
|    Cairo University| 1928|
|   Hoffmann-La Roche| 1828|
+--------------------+-----+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import substring

# 1 - Creates compStudies using the Completion column from ctDF and extracts the "Completed" results 
compStudies=ctDF.select(compCol).filter(ctDF.Status == stats)

# 2 - Creates a dictionary for months 
months = [{"months": "Jan", "Values": 1},
          {"months": "Feb", "Values": 2},
          {"months": "Mar", "Values": 3},
          {"months": "Apr", "Values": 4},
          {"months": "May", "Values": 5},
          {"months": "Jun", "Values": 6},
          {"months": "Jul", "Values": 7},
          {"months": "Aug", "Values": 8},
          {"months": "Sep", "Values": 9},
          {"months": "Oct", "Values": 10},
          {"months": "Nov", "Values": 11},
          {"months": "Dec", "Values": 12}]

# 3 - Creates a monthsDF using the months dictionary
monthsDF = spark.createDataFrame(months)
#monthsDF.show()

# 4 - Creates monYear separating the months and years from compStudies values
monYear = compStudies.select(substring('Completion', 1, 3).alias('month'),substring('Completion', 5,4).alias('year'))
#monYear.show()

# 5 - Extracts the results from ctYear, groups them by month and count the frequency for each month
freqStudies = monYear.filter(substring('Completion', 5, 4) == ctYear).groupBy('month').count()

# 6 - Inner joins the ordered months from monthsDF with the month column from monYear and selects the frequency of them
ordmonth = freqStudies.join(monthsDF, monthsDF.months == monYear.month, 'inner').select('month','count').withColumnRenamed('count', 'Frequency')

ordmonth.show()

+-----+---------+
|month|Frequency|
+-----+---------+
|  Jan|     1131|
|  Feb|      934|
|  Mar|     1227|
|  Apr|      967|
|  May|      984|
|  Jun|     1094|
|  Jul|      819|
|  Aug|      700|
|  Sep|      528|
|  Oct|      187|
+-----+---------+



In [0]:
from bokeh.models import FactorRange
from bokeh.plotting import figure
from bokeh.io import output_file, show
from bokeh.transform import dodge
from bokeh.embed import components, file_html
from bokeh.resources import CDN

# 1 - Chooses the graph type
output_file("mixed.html")

# 2 - Defines the months for X axis 
months = [ "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# 3 - Configure the bar pgraph
p = figure(x_range=FactorRange(*months), height=250,
           toolbar_location=None, tools="")

# 4 - Defines the values for Y axis
x = [ 1131, 934, 1227, 967, 984, 1094, 819, 700, 528, 187]
p.vbar(x=months, top=x, width=0.9, alpha=0.5)

# 5 - Configures the line graph
p.line(x=["Jan", "Feb", "Mar", "May", "Jun", "Aug", "Oct"], y=[1131, 934, 1227, 984, 1094, 700, 187], color="red", line_width=2)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

html =file_html(p, CDN, "my_plot1")

displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 my_plot1